<a href="https://colab.research.google.com/github/naqi72/Finetuning_TTS_Model/blob/main/English/Model_Quantization_English.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Post-Training Quantization (PTQ) to reduce model size and improve inference speed.



In [ ]:
import torch
from transformers import SpeechT5ForTextToSpeech

# Load the fine-tuned SpeechT5 model
model = SpeechT5ForTextToSpeech.from_pretrained("path/to/your/fine-tuned-model")

# Apply dynamic quantization
quantized_model = torch.quantization.quantize_dynamic(
    model,  # Model to quantize
    {torch.nn.Linear},  # Layers to quantize (e.g., Linear)
    dtype=torch.qint8  # Data type for quantization
)

# Save the quantized model
quantized_model_path = "quantized_speechT5_model.pth"
torch.save(quantized_model.state_dict(), quantized_model_path)

print("Quantization completed. Model saved.")


Fast Inference using Pruning



In [ ]:
import torch.nn.utils.prune as prune

# Prune 30% of the weights in the linear layers
for name, module in model.named_modules():
    if isinstance(module, torch.nn.Linear):
        prune.l1_unstructured(module, name='weight', amount=0.3)

# Save the pruned model
pruned_model_path = "pruned_speechT5_model.pth"
torch.save(model.state_dict(), pruned_model_path)

print("Pruning completed. Model saved.")


Inference Speed Testing on CPU/GPU and Edge Devices



In [ ]:
import time

# Load quantized or pruned model
model.load_state_dict(torch.load(quantized_model_path))

# Prepare input text
text = "I will use an API with OAuth and CUDA to train the LLM model on a GPU."
inputs = processor(text=text, return_tensors="pt")

# Measure inference time
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
inputs = {k: v.to(device) for k, v in inputs.items()}

start_time = time.time()
with torch.no_grad():
    speech = model.generate_speech(inputs["input_ids"], speaker_embeddings, vocoder=vocoder)
end_time = time.time()

inference_time = end_time - start_time
print(f"Inference time: {inference_time:.4f} seconds")


Evaluation: Trade-off between Model Size and Quality Quantized Model

In [ ]:
import os

# Check model sizes
original_size = os.path.getsize("path/to/your/fine-tuned-model.pth")
quantized_size = os.path.getsize(quantized_model_path)
pruned_size = os.path.getsize(pruned_model_path)

print(f"Original Model Size: {original_size / 1e6:.2f} MB")
print(f"Quantized Model Size: {quantized_size / 1e6:.2f} MB")
print(f"Pruned Model Size: {pruned_size / 1e6:.2f} MB")

# MOS comparison (sample)
print("MOS Scores:")
print("Original Model: 4.2/5")
print("Quantized Model: 4.0/5")
print("Pruned Model: 3.9/5")

